In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import Models.Corn_USA_Yield as cy

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [35]:
Model_Data=cy.Build_Model()
Model_Data.keys()

dict_keys(['Yield', 'Planting', 'Silking', 'w_w_df_all'])

In [36]:
Model_Data['Silking']

,week_ending,Value
272,1985-06-30,4
273,1985-07-07,11
274,1985-07-14,28
275,1985-07-21,53
276,1985-07-28,77
...,...,...
2,2021-07-11,26
3,2021-07-18,56
4,2021-07-25,79
5,2021-08-01,91
